In [ ]:
from langchain.chains import SQLDatabaseChain
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase

In [ ]:
from langchain.prompts.prompt import PromptTemplate

sql_dk_co2_db = SQLDatabase.from_uri("sqlite:///data/dk_co2_emission.db",sample_rows_in_table_info=2)

_DEFAULT_TEMPLATE = """Given a list of ingredients, extract the main ingredients from the list and create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.

Solve the task using the following steps:
- Query all ingredients in a single query. 
- In the query, remove all non-ingredient words. 
- Match the SQLResult to the list of ingredients based on preparation and type. 
- Return the Answer in the following format: ''ingredient': X kg CO2e / kg'. 
- If the ingredient is not found in the database, return '?'. 

Use the following format:
Ingredients: "Ingredients here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Begin! 

Remember to query all ingredients. 

Ingredients:
150 g red lentils
1 can of chopped tomatoes
2 cubes of vegetable bouillon
1 tin of tomato concentrate (140 g)
1 tbsp. lemon juice
1. tbsp. chili powder
1 starfruit

SQLQuery: SELECT Name, Total_kg_CO2_eq_kg FROM dk_co2_emission WHERE 
          Name LIKE '%tomato%' OR 
          Name LIKE '%lentil%' OR 
          Name LIKE '%bouillion%' OR 
          Name LIKE '%juice%' OR
          Name LIKE '%lemon%' OR 
          Name LIKE '%chili%' OR
          Name LIKE '%starfruit%'

SQLResult: [('Tomato, ripe, raw, origin unknown', 0.7), ('Green lentils, dried', 1.78), 
            ('Tomatojuice, canned', 1.26), ('Tomato, peeled, canned', 1.26), 
            ('Tomato paste, concentrated', 2.48), ('Red lentils, dried', 1.78),
            ('Ice, popsickle, lemonade', 1.15), ('Lemon, raw', 0.94),
            ('Apple juice', 1.64),('Bouillon, chicken, prepared', 0.38), 
            ('Bouillon, beef, prepared', 0.52), ('Pepper, hot chili, raw', 1.02), 
            ('Pepper, hot chili, canned', 1.54),  
            ]
            
Answer: 
Red lentils: 1.78 kg CO2e / kg
Chopped tomatoes: 1.26 kg CO2e / kg
Vegetable bouillon: 0.38 kg CO2e / kg (closest was chicken bouillon)
Tomato concentrate (140 g): 2.48 kg CO2e / kg (closest was tomato paste)
Lemon juice: 0.94 kg CO2e / kg (Closest was Lemon, raw)
Starfruit: ? (Not found in database)

Ingredients: 
{input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
llm = ChatOpenAI(
    temperature=0,
)
db_chain = SQLDatabaseChain(llm=llm, database=sql_dk_co2_db, verbose=True, prompt=PROMPT, top_k=50)

In [ ]:
prompt1 = """150 g red lentils
             1 can of chopped tomatoes,
             1 large onion, 
             1 pepper
             2 cubes of vegetable bouillon
             3 cloves garlic, pressed
             1 tin of tomato concentrate (140 g),
             1 can of coconut milk (400 ml),
             1 tbsp. lemon juice,
             1 tsp. sugar,
             ½ teaspoon chili powder,
             ½ teaspoon ground cumin,
             Possibly. 1/4 tsp. smoked paprika,
             A large handful of fresh basil, chopped
             (or 2 tsp dried basil)
             """

prompt2 = """3 tbsp olive oil
            0.50 red chili, remove seeds and membrane and cut into thin slices
            400 g of pasta
            2 cloves of garlic, finely chopped
            1 tin of peeled tomatoes
            1 teaspoon balsamic vinegar
            1 teaspoon of sugar
            1 organic lemon, finely grated peel from here
            2 handfuls of fresh basil
            salt
            black pepper, freshly ground
             """

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:

    answer1 = db_chain.run(prompt2)
    
    print("Total Tokens:", cb.total_tokens)
    print("Total prompt tokens:", cb.prompt_tokens)

## Create weight estimator llm

In [ ]:
TEMPLATE = """
Given a list of ingredients, estimate the weights in kilogram for each ingredient.

The following recalculations can be used to estimate the weight of each ingredient:
1 can = 400 g
1 cube of bouillon/stock = 4 g
1 large onion = 285 g
1 medium onion = 170 g
1 small onion = 115 g
1 pepper = 150 g
1 tin of tomato concentrate = 140 g
1 tbsp. = 15 g
1 tsp. = 5 g
1 potato = 170 - 300 g
1 carrot = 100 g
1 lemon = 60 g

Use the following format:
Ingredients: "Ingredients here"
Answer: "Final answer here"

Begin!

Ingredients: 
1 can of chopped tomatoes
2 large potatoes
3 teaspoons of sugar
200 g of pasta
500 ml of water
250 gram of minced meat
0.5 cauliflower
1 organic lemon
salt
pepper

Answer:
1 can of chopped tomatoes: 0.4 kg
2 large potatoes: 0.6 kg
3 teaspoons of sugar: 0.15 kg
200 g of pasta: 0.2 kg
500 ml of water: 0.5 kg
250 gram of minced meat: 0.25 kg
0.5 cauliflower: 0.250 kg (estimated by LLM model)
1 organic lemon: 0.06 kg
salt: negligible
pepper: negligible


Ingredients: 
{input}
"""

llm = ChatOpenAI(
    temperature=0
)

prompt = PromptTemplate(
    input_variables=["input"],
    template=TEMPLATE,
)

weight_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)



In [ ]:
with get_openai_callback() as cb:

    answer2 = weight_chain.run(prompt2)
    print("Total Tokens:", cb.total_tokens)
    print("Total prompt tokens:", cb.prompt_tokens)

In [ ]:
print(answer2)

## Calculate total CO2 emission

In [ ]:
from food_co2_estimator.utils.openai_model import get_model
from langchain import LLMMathChain
from langchain.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If you execute code, you must print out the final answer using the print function. You MUST use the python package numpy to answer your question. You must import numpy as np.

You are provide two lists of ingredients where one list provides the CO2 emission per kilogram of the ingredient, and the other list provides the amount of the ingredient in kilogram. You must calculate the total CO2 emission of the recipe.
The lists are separated by a "---".

Some ingredients are not found in the database and are shown with: ?. Transform the ? to np.nan when calculating total CO2 emission.
Ingredients not included in the calculation should be listed after the answer.

Question: ${{List of ingredients}}
```python
${{Code that prints what you need to know}}
print(${{code}})
```
```output
${{Output of your code}}
```
Answer: ${{Answer}} kg CO2e for the recipe \n ${{Ingredients not included in the calculation}}

Begin.

Question: 
Red lentils: 1.78 kg CO2e/kg
Chopped tomatoes: 1.26 kg CO2e/kg 
Chili powder: ?
---
Red lentils 0.3 kg
Chopped tomatoes 0.2 kg
Chili powder: 0.001 kg

```python
import numpy as np
print(np.nansum(np.array([1.78, 1.26,np.nan]) * np.array([0.3, 0.2, 0.001]))
```
```output
0.786
```
Answer: 0.786 kg CO2e for the recipe \n Chili powder is not included in the calculation.

Question: {question}"""

PROMPT = PromptTemplate(input_variables=["question"], template=_PROMPT_TEMPLATE)

llm = LLMMathChain(llm=get_model())

In [ ]:
with get_openai_callback() as cb:
    answer3 = llm.run(f"{answer1}\n --- \n{answer2}")
    
    print("Total Tokens:", cb.total_tokens)
    print("Total prompt tokens:", cb.prompt_tokens)
    